# Summarize WSF

The World Settlement Footprint was officially launched in November 2021, [see the announcement here for more details](https://www.esa.int/Applications/Observing_the_Earth/Mapping_our_human_footprint_from_space).

GOST have downloaded the evolution dataset, generated a VRT combining the ~5000 tiles, and uploaded to our private AWS bucket (wbgdecinternal-ntl). That dataset will be used to summarize change in the built up area in a variety of urban areas

In [ ]:
import sys, os, importlib
import rasterio

import pandas as pd
import geopandas as gpd

try:
    import GOSTRocks.rasterMisc as rMisc    
    from GOSTRocks.misc import tPrint
except:
    print("Cannot find GOSTRocks")


In [ ]:
wsf_evolution = 's3://wbgdecinternal-ntl/GIS_Data/GLOBAL/WSFEvolution/WSF_Evolution.vrt'
urban_extents = '/home/public/Data/GLOBAL/URBAN/GHS/GHS_STAT_UCDB2015MT_GLOBE_R2019A/GHS_STAT_UCDB2015MT_GLOBE_R2019A_V1_2.gpkg'
out_folder = "/home/wb411133/data/Projects/SD_FLAGSHIP/Data/GHS_URB_RES"

#Columns in urban database to keep in output - includes ID column and names
inD_cols = ['ID_HDC_G0','CTR_MN_NM','CTR_MN_ISO','UC_NM_MN', 'UC_NM_LST'] 
unq_vals = [0] + list(range(1985, 2016))

In [ ]:
inR = rasterio.open(wsf_evolution)
inD = gpd.read_file(urban_extents)

In [ ]:
res = rMisc.zonalStats(inD, inR, rastType='C', unqVals=unq_vals)

named = pd.DataFrame(res, columns=[f'c_{x}' for x in unq_vals])
for col in inD_cols:
    named[col] = inD[col]

named.to_csv(os.path.join(out_folder, "WSF_Evolution.csv"))

# Map some results

In [ ]:
# Select a single city in the dataset
sel_city = inD.loc[inD['UC_NM_MN'] == "Washington D.C."]
sel_city

In [ ]:
# I cannot get ipyleaflet to work on the JNB

from ipyleaflet import Map, basemaps, basemap_to_tiles

c = sel_city.unary_union.centroid
m = Map(
    basemap=basemap_to_tiles(basemaps.OpenStreetMap.BlackAndWhite),
    center=(c.x,c.y),
    zoom=8
)

m